In [5]:
!pip install unsloth
!pip install -U "huggingface_hub[cli]"

In [ ]:
%%shell
huggingface-cli login --token YOUR_TOKEN


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `legal` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `legal`


In [7]:
import torch
from datasets import load_dataset
from trl import SFTTrainer

In [8]:
from unsloth import FastLanguageModel
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Llama-3.2-3B-bnb-4bit',
     max_seq_length=2048,  #maximum number of tokens the model can process in a single input sequence
    load_in_4bit=True,
    dtype=None
)
model = FastLanguageModel.get_peft_model(
 model,
 r = 16, # LoRA rank
 target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
 lora_alpha = 16,
 lora_dropout = 0, # Optimized at 0
 bias = "none", # No additional bias terms
 use_gradient_checkpointing = "unsloth", # Gradient checkpointing to save memory
 random_state = 3407,
 use_rslora = False, # Rank stabilized LoRA, can be enabled for stability
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
dataset = load_dataset('opennyaiorg/aalap_instruction_dataset',split="train")
import pandas as pd
dataset=pd.DataFrame(dataset)
# dataset.columns

README.md:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

(…)-00000-of-00001-5242b1cfcb40f3dc.parquet:   0%|          | 0.00/92.9M [00:00<?, ?B/s]

(…)-00000-of-00001-c4653fdde527ae78.parquet:   0%|          | 0.00/4.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1094 [00:00<?, ? examples/s]

In [10]:
dataset = dataset[dataset["task"].str.contains("argument_generation")]
dataset.head()

,input_text,system_prompt,user_prompt,output_text,task,combined_input_prompt
9,"\nfacts:\n""""""This appeal by the Revenue under ...",You are acting as an Indian lawyer in Bombay H...,None,Argument 1: Profit arising from the transfer o...,argument_generation___petitioner,<s> [INST] <<SYS>> You are acting as an Indian...
15,"\nfacts:\n"""""" Jagadisan, J. \n 1.\nThis is an...","As an attorney in Madras High Court, you have ...",None,Counter Arguments:\n\nArgument 1: Jurisdiction...,argument_generation___respondent,<s> [INST] <<SYS>> As an attorney in Madras Hi...
30,"\nfacts:\n"""""" S.N. Shankar, J.\n \n (1) This a...",Your role is that of an Indian lawyer practici...,None,Argument 1: Violation of Section 13(2) of the ...,argument_generation___petitioner,<s> [INST] <<SYS>> Your role is that of an Ind...
36,"\nfacts:\n""""""\n\n\n\nJUDGMENT:\nCIVIL APPELLAT...",You are an Indian lawyer located in Supreme Co...,None,Argument 1: Violation of Section 4 of the Cent...,argument_generation___petitioner,<s> [INST] <<SYS>> You are an Indian lawyer lo...
56,"\nfacts:\n""""""Chapter III of the said Act deals...",In Supreme Court of India. You assume the role...,None,Counter Argument 1: Validity of the proviso ap...,argument_generation___respondent,<s> [INST] <<SYS>> In Supreme Court of India. ...


In [11]:
dataset["input_text"] = dataset["input_text"].str.replace(r'[\n"""[\]/]', '', regex=True)
dataset.head(1)

,input_text,system_prompt,user_prompt,output_text,task,combined_input_prompt
9,facts:This appeal by the Revenue under Section...,You are acting as an Indian lawyer in Bombay H...,None,Argument 1: Profit arising from the transfer o...,argument_generation___petitioner,<s> [INST] <<SYS>> You are acting as an Indian...


In [12]:
from unsloth.chat_templates import get_chat_template,standardize_sharegpt

# ds.drop_column("conversations")

ds = dataset.copy()

def format_prompts(row):
    return[
            {"content": row['system_prompt'], "role": "system"},
            {"content": row['input_text'], "role": "user"},
            {"content": row['output_text'], "role": "assistant"}
        ]

# Apply the corrected format
ds["conversations"] = ds.apply(format_prompts, axis=1)

In [13]:
ds["conversations"]

,conversations
9,[{'content': 'You are acting as an Indian lawy...
15,[{'content': 'As an attorney in Madras High Co...
30,[{'content': 'Your role is that of an Indian l...
36,[{'content': 'You are an Indian lawyer located...
56,[{'content': 'In Supreme Court of India. You a...
...,...
20693,[{'content': 'In Kerala High Court. You assume...
20725,[{'content': 'You are operating as an Indian l...
20729,[{'content': 'As an attorney in Patna High Cou...
20735,[{'content': 'You are an Indian lawyer practic...


In [14]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
)

ds["text"] = ds["conversations"].apply(
    lambda convo: tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
)

In [15]:
ds["text"].head()

,text
9,<|begin_of_text|><|start_header_id|>system<|en...
15,<|begin_of_text|><|start_header_id|>system<|en...
30,<|begin_of_text|><|start_header_id|>system<|en...
36,<|begin_of_text|><|start_header_id|>system<|en...
56,<|begin_of_text|><|start_header_id|>system<|en...


In [22]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from datasets import Dataset

dataset = Dataset.from_pandas(ds)

trainer = SFTTrainer(
 model = model,
 tokenizer = tokenizer,
 train_dataset = dataset,
 dataset_text_field = "text",
 max_seq_length = 2048,
 data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
 dataset_num_proc = 2,
 packing = False,
 args = TrainingArguments(
 per_device_train_batch_size = 2,
 gradient_accumulation_steps = 4,
 warmup_steps = 10,
 max_steps = 45,
 learning_rate = 2e-4,
 fp16 = not is_bfloat16_supported(),
 bf16 = is_bfloat16_supported(),
 logging_steps = 1,
 optim = "adamw_8bit",
 weight_decay = 0.01,
 lr_scheduler_type = "linear",
 seed = 3407,
 output_dir = "outputs",
 ),
)


Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1142 [00:00<?, ? examples/s]

In [23]:
from unsloth.chat_templates import train_on_responses_only
import torch

# Ensure 'labels' column exists and is correctly formatted
def add_labels(examples):
    return {"labels": examples["text"]}

dataset = dataset.map(add_labels)

# Convert 'labels' to tensors
def convert_labels_to_tensors(examples):
    if isinstance(examples["labels"], list):
        return {"labels": [torch.tensor(label, dtype=torch.long) for label in examples["labels"]]}
    return examples

dataset = dataset.map(convert_labels_to_tensors)

# Train model


Map:   0%|          | 0/1142 [00:00<?, ? examples/s]

Map:   0%|          | 0/1142 [00:00<?, ? examples/s]

In [24]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/1142 [00:00<?, ? examples/s]

In [25]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,142 | Num Epochs = 1 | Total steps = 45
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,0.405400
2,0.414600
3,0.433900
4,0.440000
5,0.389900
6,0.309900
7,0.309500
8,0.294900
9,0.451200
10,0.333000


In [34]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content":
     """
      H.N. Seth, J.
1.
The assessee, U.P. Agro Industrial Corporation Ltd., Lucknow (hereinafter referred to as "the Agro Corporation") was set up in the month of March, 1947, as a body registered under the Indian Companies Act.
Capital of the Agro Corporation was subscribed equally by the Govt. of India and the Govt. of Uttar Pradesh.
The State Govt. vide its order No. 1748/XII-J-246/67, dated April 27, 1968, transferred possession over the Agricultural Workshop, Lucknow, and Pilot Project, Karimganj to the Agro Corporation in consideration of Rs. 44,07,589 (Rs. 40,00,000 worth of equity shares and Rs. 4,07,589 paid in cash).
During the assessment proceedings, the Agro Corporation, inter alia, claimed allowance by way of depreciation under Section 32 of the I.T. Act, in respect of the buildings taken over by it from the State Govt.
3. The ITO disallowed the aforesaid claim for depreciation on the ground that the properties in respect of which the claim had been made were immovable properties.
Since no sale deed had been executed by the State Govt. till then, the Agro Corporation did not become its owner.
It was accordingly not entitled to claim any depreciation allowance in respect of such property.

4. In appeal before the AAC, it was contended on behalf of the Agro Corporation
that even though the U.P. Govt. had not transferred the immovable property by a registered deed, the properties for all practical purposes belonged to it.
The Agro Corporation was the beneficial and equitable owner of the property and was in connection with its assessment entitled to claim depreciation on it.
The AAC, held that in the absence of a registered sale deed, the properties in question which were immovable properties did not stand transferred to the Agro Corporation.
As the properties were not owned by it, it was not entitled to claim depreciation in respect thereof.
5. In second appeal by the Agro Corporation, the Income-tax Appellate Tribunal observed that for claiming depreciation under Section 32 it was necessary for the assessee to fulfil the following two conditions :
(1) that the assets in respect of which the depreciation is claimed by the assessee must be owned by him ; and (2) that those assets must have been used by the assessee for the purposes of his business.
6. So far as the second condition was concerned, it was not disputed that the property in respect of which depreciation had been claimed by the assessee was being used for its business purposes.
The Tribunal further pointed out that the property in respect of which the Agro Corporation had claimed allowance by way of depreciation consisted of land and buildings.
It was entitled to claim depreciation only in respect of the building and was not entitled to lay such claim in respect of the land.
As figures with regard to the value of the building and land were not separately available, it sent the case back to the ITO for ascertaining the value of the building on which alone the Agro Corporation was entitled to claim depreciation.

sections of law:
['Section 32(1) of Income Tax Act', 'proviso to Section 53A of Transfer of Property Act', 'Section 22 of Income Tax Act', 'Section 10(2)(vi) of Income Tax Act', 'Section 34 of Income Tax Act', 'Section 9(1) of Income Tax Act', 'Section 53A of Transfer of Property Act', 'Section 32 of Income Tax Act', 'Section 54 of Transfer of Property Act', 'Section 9 of Income Tax Act']

Issues:
['Ownership of properties transferred by the State Government to the Agro Corporation', 'Entitlement of the Agro Corporation to claim depreciation on the transferred properties', 'Separate valuation of the building and land for claiming depreciation']

     """
     },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=2048, use_cache=True, temperature=1.5, min_p=0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n\n      H.N. Seth, J.\n1.\nThe assessee, U.P. Agro Industrial Corporation Ltd., Lucknow (hereinafter referred to as "the Agro Corporation") was set up in the month of March, 1947, as a body registered under the Indian Companies Act.\nCapital of the Agro Corporation was subscribed equally by the Govt. of India and the Govt. of Uttar Pradesh.\nThe State Govt. vide its order No. 1748/XII-J-246/67, dated April 27, 1968, transferred possession over the Agricultural Workshop, Lucknow, and Pilot Project, Karimganj to the Agro Corporation in consideration of Rs. 44,07,589 (Rs. 40,00,000 worth of equity shares and Rs. 4,07,589 paid in cash).\nDuring the assessment proceedings, the Agro Corporation, inter alia, claimed allowance by way of depreciation under Section 32 of the I.T. Act, in respect of the buildings 